In [15]:
import torch
from torch import nn

In [16]:
class myGraphSAINT(nn.Module):
    def __init__(self):
        super(my_model, self).__init__()
    
    def forward(self, x):
        return x
    
    def sampleGraph(self, graph):
        pass

In [17]:
x = torch.ones(3)
model = my_model()
print(model(x))

tensor([1., 1., 1.])
